## 用不同方式寫不同NN

### initial preparation
#### keras backend 可以設置以保持執行的一致性

In [1]:
%env KERAS_BACKEND = tensorflow

env: KERAS_BACKEND=tensorflow


In [2]:
%matplotlib inline 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import keras
from ipywidgets import interact

Using TensorFlow backend.


In [3]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras.optimizers import SGD, Adam
from keras.datasets import mnist
from keras.utils import to_categorical

from sklearn.model_selection import train_test_split

## Load data

In [4]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
print ("共 %d 筆資料，每筆資料為 %d x %d " %x_train.shape)

共 60000 筆資料，每筆資料為 28 x 28 


## 格式整理

In [6]:
x_train0 = x_train
y_train0 = y_train
x_test0 = x_test
y_test0 = y_test

In [7]:
x_train = x_train0.reshape((x_train0.shape[0], 28*28))
x_test = x_test0.reshape((x_test0.shape[0], 28*28))

In [8]:
x_train -= x_train.min()
x_train = x_train/x_train.max()

In [9]:
print (x_train.min(), x_train.max())

0.0 1.0


In [10]:
x_test -= x_test.min()
x_test = x_test/x_test.max()

In [11]:
print (x_test.min(), x_test.max())

0.0 1.0


In [12]:
y_train_num = to_categorical(y_train, num_classes=10)
y_test_num = to_categorical(y_test, num_classes=10)

## 回顧 Sequential

In [13]:
model = Sequential()

model.add(Dense(500, input_dim=784))
model.add(Activation("sigmoid"))
model.add(Dense(500))
model.add(Activation("sigmoid"))
model.add(Dense(10))
model.add(Activation("softmax"))

In [14]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 500)               392500    
_________________________________________________________________
activation_1 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_2 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                5010      
_________________________________________________________________
activation_3 (Activation)    (None, 10)                0         
Total params: 648,010
Trainable params: 648,010
Non-trainable params: 0
_________________________________________________________________


## 另一種使用Sequential建NN方式

In [15]:
model.layers

###  用 model  list 打包

In [20]:
first_layer = [Dense(500, input_dim=784, name="Dense_1"), Activation("sigmoid")]
second_layer = [Dense(500, name="Dense_2"), Activation("sigmoid")]
last_layer = [Dense(10, name="Dense_3"), Activation("softmax")]

In [21]:
first_layer[0].trainable=True

In [22]:
model = Sequential(first_layer+second_layer+last_layer)

In [23]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Dense_1 (Dense)              (None, 500)               392500    
_________________________________________________________________
activation_7 (Activation)    (None, 500)               0         
_________________________________________________________________
Dense_2 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_8 (Activation)    (None, 500)               0         
_________________________________________________________________
Dense_3 (Dense)              (None, 10)                5010      
_________________________________________________________________
activation_9 (Activation)    (None, 10)                0         
Total params: 648,010
Trainable params: 648,010
Non-trainable params: 0
_________________________________________________________________


#### 用 list 好處為不用每次改整個 model

### 轉移學習：將以訓練之 model 更換資料學習

In [43]:
All_except_last = first_layer + second_layer
last_layer = [Dense(2), Activation("softmax")]
model_num = Sequential(All_except_last + last_layer)

In [44]:
model_num.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 500)               392500    
_________________________________________________________________
activation_7 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_8 (Activation)    (None, 500)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 1002      
_________________________________________________________________
activation_17 (Activation)   (None, 2)                 0         
Total params: 644,002
Trainable params: 644,002
Non-trainable params: 0
_________________________________________________________________


In [45]:
model_num.compile(loss="categorical_crossentropy", optimizer="sgd")

## Trainable = False

In [54]:
All_except_last[0].trainable = False


## Functional API

In [55]:
from keras.models import Model
from keras.layers import Input